# Case Study: Cultural Domain Analysis (CDA)

Cultural domain analysis (CDA) is the study of how people in groups think about lists of terms that somehow go together and how this thinking differs between groups.

Anthropologists, ethnographers, psychologists, and sociologists use CDA to understand semantic mindscapes of social, ethnic, religious, professional, and other groups. 

You will see how to convert natural language units into terms and build, analyze, and interpret a semantic network reflecting the interests of the fans of <b>The Good Wife</b>, a CBS TV show. Hopefully, you will be able to extend the same approach to other shows, other websites, and other tag <b>corpora</b>.

Excerpt From: Dmitry Zinoviev. “Complex Network Analysis in Python”

In [1]:
# This program performs CDA for a LiveJournal community
import urllib.request, os.path, pickle # Download and cache
import nltk # Convert text to terms
import networkx as nx, community # Build and analyze the network
import pandas as pd, numpy as np # Data science power tools

A **term** is a unit of CDA. It can be a word, a word group, a word stem, or even an emoticon (emoji). CDA looks into similarities between terms that are shared among a reasonably homogeneous group of people. So, you need to find a reasonably homogeneous group of individuals, a list of terms, and a way to assess their similarity.

A great source of semantic data is <b>LiveJournal</b> (LJ) — a collection of individual and communal blogs with elements of a massive online social network. LiveJournal has an open, easily accessible application programming interface (API), and encourages the use of public data for research.

The URLs of user/community profiles, interest lists, and friend lists have a regular structure. If thegoodwife_cbs is the name of a community (you will use it in the rest of the study), then https://thegoodwife-cbs.livejournal.com/profile/ is the URL of the community profile, https://www.livejournal.com/misc/fdata.bml?user=thegoodwife_cbs&comm=1 is the friend list, and https://www.livejournal.com/misc/interestdata.bml?user=thegoodwife_cbs is the interest list.

The rest of the community membership list here is a two-column table. The first column represents some subtle aspects of membership types (“P” for individual users, “C” for communities, “<” for “friends,” “>” for “friends-of”); the second column has user names. To keep your code modular, write a function <b>download(domain_name)</b> that takes care of this and similar tables.

In [2]:
# Download interest data from the domain_name community on
# LiveJournal, convert interests to tags, create a domain DataFrame
def download(base, domain_name):
    
    # Convert the interest table into a two-column DataFrame. 
    members_url = "{}/fdata.bml?user={}&comm=1".format(base, domain_name) 
    members = pd.read_table(members_url, sep=" ", 
                            comment="#", names=("direction", "uid"))

    # Prepare a *WordNet* lemmatizer *wnl* - a tool for converting words into lemmas — and 
    # a list of stopwords *stop*, extended to include “&.” You will need this list 
    # to eliminate too frequent words. Coerce the standard list of stopwords into a set 
    # for faster lookup, because list lookups in Python are notoriously slow.
    wnl = nltk.WordNetLemmatizer()
    
    # uncomment command below if you need to download resource 'corpora/stopwords.zip/stopwords/'
    # nltk.download()
    
    stop = set(nltk.corpus.stopwords.words('english')) | set(('&'))
    
    # Set up an empty list accumulator term_vectors. At the end of the loop, 
    # it becomes a list of term vectors—raw material for the term matrix. 
    term_vectors = []
    
    # Loop through all unique user names in the community, 
    # because you need the URLs of their interest lists.
    for user in members.uid.unique():
        print("Loading {}".format(user)) # Progress indicator
        user_url = "{}/interestdata.bml?user={}".format(base, user)
        
        # Obtain an interest list *raw_interests* as a Python list of strings for each distinct community 
        # member in the try-except block. If the URL fails to open (for any reason beyond your control), 
        # the script does not crash but politely informs the programmer of the failure and proceeds 
        # to the next user. Same thing happens when the user has no interests or does not exist at all. 
        # If everything goes fine, decode and strip each string of trailing white spaces. 
        # LJ interest lists are always in the lower case, but if they were not, a call to lower ensures 
        # that in the rest of the script you compare apples to apples. 
        try: 
            with urllib.request.urlopen(user_url) as source:
                raw_interests = [line.decode().lower().strip() 
                                 for line in source.readlines()]
        except:
            print("Could not open {}".format(user_url)) # Error message
            continue

        if raw_interests[0] == '! invalid user, or no interests':
            continue

        # Split each non-empty, non-commented interest into individual words with *nltk.wordpunct_tokenize*. 
        # There may be more than one word in an interest description, and some or all of these words 
        # may be forms of other words (as in “chicks”—“chick”). Text analysis practitioners preach different 
        # ways of handling word forms. Some suggest to leave the forms alone and treat them as words on their own. 
        # Some advocate lemmatizing or stemming: reducing a form to its lemma (the standard representation 
        # of the word) or to the stem (the smallest meaningful part of the word to which affixes can be attached). 
        # A lemmatizer reduces “programmers” to one “programmer,” and a stemmer, depending on its zeal, 
        # yields a “programm” or even a “program.” Let’s follow the lemmatizing crowd. 
        # Furthermore, almost everyone agrees that certain words (such as “a,” “the,” and “and”) 
        # should never be counted at all. Remove them.
        interests = [" ".join(wnl.lemmatize(w)
                              for w in nltk.wordpunct_tokenize(line)[2:] 
                              if w not in stop)
                     for line in raw_interests 
                     if line and line[0] != "#"]
        
        # As a result of lemmatization, stemming, and stopword elimination, a term list may end up having 
        # duplicates (e.g., “the chicks” and “a chick” may both become “chick”). Convert each term list 
        # into a set. Surely, sets have no duplicates.
        interests = set(interest for interest in interests if interest)
        
        # Your goal is to produce a term vector model (TVM) — a table where rows are terms and columns 
        # are community members. In the Pandas language, this table is known as DataFrame, 
        # and its columns are known as Series. Transform a term list to a Series, where the individual 
        # terms become the Series index, and all values are set to 1s, like this:
        #
        #       print(term_vectors)
        #       <= shiny!        1
        #       +5 sexterity     1
        #       big damn hero    1
        #       …more terms…
        #       Name: twentyplanes, dtype: float64
  
        term_vectors.append(pd.Series(index=interests, name=user).fillna(1))
    
    # Finally, join all *Series* into a *DataFrame*. This operation involves the mystery of data alignment, 
    # whereby all participating *Series* are stretched vertically to have their row labels (terms!) aligned. 
    # Such stretching almost inevitably produces empty cells in the frame. Fill them up with zeros: 
    # an empty cell in the row A and column B signals that the term A was not on the list B. 
    # The resulting variable *DataFrame* has 12,437 rows and adequately represents the cultural domain 
    # of LiveJournal users interested in the CBS show "The Good Wife".
    return pd.DataFrame().join(term_vectors, how="outer").fillna(0)\
        .astype(int)#(9)

In [3]:
LJ_BASE = "http://www.livejournal.com/misc"
DOMAIN_NAME = "thegoodwife_cbs"

# we have to create directory cache to store all downloaded data into it

cache_d = "cache/" + DOMAIN_NAME + ".pickle"
if not os.path.isfile(cache_d):
    domain = download(LJ_BASE, DOMAIN_NAME)
    if not os.path.isdir("cache"):
        os.mkdir("cache")
    with open(cache_d, "wb") as ofile:
        pickle.dump(domain, ofile)
else:
    with open(cache_d, "rb") as ifile:
        domain = pickle.load(ifile)

Loading emploding
Loading poocat
Loading tellitslant
Loading sakurazukakami
Loading quixoticfish
Loading commonlogic
Loading returnofpiper
Loading fatascribunda
Loading ponka
Loading beck_liz
Loading streetrat
Loading belle1446
Loading peonelli
Loading beyondallreason
Loading mirareeves
Loading kseda
Loading goldenhair
Loading angua9
Loading phrenitis
Loading jools_joyce
Loading pouletinbondage
Loading deborah_judge
Loading only_more_love
Loading solookup
Loading jlightstar
Loading survived_it_all
Loading empty_marrow
Loading jenncho
Loading anjali_organna
Loading michellek
Loading sporkythespaz
Loading stabbim
Loading raindrops888
Loading demishock
Loading sharz
Loading firedawn
Loading andreva
Loading littlehutt
Loading squaringkarma
Loading darsfebruary
Loading yuffie9852
Loading spyglass_
Loading shades_children
Loading fireworkfreeway
Loading mica_chan
Loading jhsc22
Loading patracoras
Loading whosmurry
Loading arabian
Loading ekroe
Loading fireflies_uk
Loading 9tails
Loading free

Loading heyheartbreak
Loading diana_dwight
Loading pecalibre
Loading oxymoron06
Loading meyrin
Loading ramblingnotions
Loading justpretend
Loading yayiegurl0307
Loading readsalot234
Loading goodwifefan
Loading youwatchusrun
Loading kalindas
Loading nessakitty821
Loading allthecities
Loading primalmusic
Loading slytherinsnitch
Loading whoredemort
Loading stormingcastles
Loading ddd_dcb
Loading seldonp38
Loading raktajinos
Loading leeleew
Loading zulabelle
Loading sedonaazhomes
Loading chinesezodiac13
Loading fandom_forever
Loading christina4172
Loading 50sfrenchmovie
Loading brooketiffany
Loading harperjohnson
Loading esotaria
Loading k8_catherine
Loading aishia
Loading freya86
Loading cyberducks
Loading moonlightrick
Loading koalathebear
Loading the_grynne
Loading einzelle
Loading draconianfreak
Loading ayumie
Loading lunalovepotter
Loading snurri
Loading bevfank
Loading pretty_cynical
Loading hellandbliss
Loading illona999
Loading ishi_chan
Loading mirageofmae
Loading raffyx
Loading w

## Build the Term Network

You could have included all 12,437 discovered terms in the graph, but some of them are mentioned only once or twice 
(which is expected, given Zipf’s law). 

Rather than wondering why the less frequently used words are in fact less frequently used, remove all rows with 
fewer than ten occurrences, but provide an option of changing the cut-off value MIN_SUPPORT in the future. 
DataFrame limited is a truncated version of domain: it has only 319 rows.

In [4]:
MIN_SUPPORT = 10
sums = domain.sum(axis=1)
limited = domain[sums >= MIN_SUPPORT]

In [5]:
print(limited)

                      poocat  tellitslant  sakurazukakami  quixoticfish  \
24                         0            0               0             0   
30 rock                    0            0               0             0   
aaron sorkin               0            0               0             0   
alan rickman               0            0               0             0   
alias                      0            0               0             0   
alicia /                   0            0               0             0   
alicia florrick            0            1               0             0   
allison janney             0            0               0             0   
american idol              0            0               0             0   
amy poehler                0            0               0             0   
angel                      0            0               0             0   
anime                      0            0               1             0   
archie panjabi           

Since you want to build a network based on co-occurrences, you can consider two terms as similar if different community members frequently use them together. Calculate the matrix of co-occurrence by matrix-multiplying the <b>limited DataFrame</b> by itself.

The resulting square <b>DataFrame cooc</b> contains the total counts of all terms on the main diagonal (suppress them by multiplying the matrix by an inverted identity matrix) and the counts of co-occurrences elsewhere (they will eventually become weighted network edges).

In [6]:
cooc = limited.dot(limited.T) * (1 - np.eye(limited.shape[0]))

In [7]:
print(cooc)

                       24  30 rock  aaron sorkin  alan rickman  alias  \
24                    0.0      5.0           3.0           0.0    6.0   
30 rock               5.0      0.0           2.0           0.0    9.0   
aaron sorkin          3.0      2.0           0.0           2.0    4.0   
alan rickman          0.0      0.0           2.0           0.0    2.0   
alias                 6.0      9.0           4.0           2.0    0.0   
alicia /              0.0      3.0           0.0           0.0    6.0   
alicia florrick       1.0      2.0           1.0           1.0    2.0   
allison janney        2.0      2.0           5.0           3.0    2.0   
american idol         1.0      2.0           0.0           0.0    2.0   
amy poehler           0.0      7.0           1.0           2.0    1.0   
angel                 6.0      8.0           4.0           0.0   14.0   
anime                 1.0      2.0           0.0           1.0    0.0   
archie panjabi        0.0      2.0           0.0   

## Slice the network

Now, you must make another painful decision: which matrix elements become edges and which get discarded? 
Choose the slicing threshold, **SLICING**, to be six. Higher **SLICING** results in many small communities. 
Lower **SLICING** results in few large communities. Six seems to be a good compromise between count and size.

The resulting matrix is very sparse (every cell represent an edge, but we agreed to have as few edges as possible!)
Stack and normalize it—essentially convert into a sparse matrix, where each row represents a significant edge and 
its weight. Since **networkx** prefers to deal with Python (rather than Pandas) data structures, convert the 
**weights** to a dictionary.

In [9]:
SLICING = 6
weights = cooc[cooc >= SLICING]
weights = weights.stack()
weights = weights / weights.max()
cd_network = weights.to_dict()
cd_network = {key:float(value) for key,value in cd_network.items()}

In [10]:
print(cd_network)

{('24', 'alias'): 0.0967741935483871, ('24', 'angel'): 0.0967741935483871, ('24', 'battlestar galactica'): 0.14516129032258066, ('24', 'buffy vampire slayer'): 0.12903225806451613, ('24', 'castle'): 0.11290322580645161, ('24', 'dexter'): 0.11290322580645161, ('24', 'friend'): 0.14516129032258066, ('24', 'fringe'): 0.12903225806451613, ('24', 'glee'): 0.0967741935483871, ('24', 'good wife'): 0.12903225806451613, ('24', "grey ' anatomy"): 0.11290322580645161, ('24', 'harry potter'): 0.14516129032258066, ('24', 'lost'): 0.16129032258064516, ('24', 'music'): 0.12903225806451613, ('24', 'ncis'): 0.0967741935483871, ('24', 'six foot'): 0.0967741935483871, ('24', 'supernatural'): 0.14516129032258066, ('24', 'vampire diary'): 0.11290322580645161, ('24', 'west wing'): 0.11290322580645161, ('24', 'x - file'): 0.11290322580645161, ('30 rock', 'alias'): 0.14516129032258066, ('30 rock', 'amy poehler'): 0.11290322580645161, ('30 rock', 'angel'): 0.12903225806451613, ('30 rock', 'arrested development

Let’s create a new empty graph, populate it with the edges from the dictionary, and update the “weight” edge attributes:

In [11]:
tag_network = nx.Graph()
tag_network.add_edges_from(cd_network)
nx.set_edge_attributes(tag_network, cd_network, "weight")

Let's add a "w" node attribute to count the number of times the corresponding tag was mentioned in the corpus

In [12]:
sizes = {key:float(value) for key,value in 
         domain.ix[tag_network].sum(axis=1).items()}
nx.set_node_attributes(tag_network, sizes, "w")

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [13]:
sizes

{'24': 19.0,
 '30 rock': 46.0,
 'aaron sorkin': 18.0,
 'alan rickman': 12.0,
 'alias': 49.0,
 'alicia /': 13.0,
 'alicia florrick': 12.0,
 'allison janney': 12.0,
 'american idol': 11.0,
 'amy poehler': 12.0,
 'angel': 42.0,
 'anime': 22.0,
 'archie panjabi': 11.0,
 'arrested development': 26.0,
 'art': 31.0,
 'asoiaf': 14.0,
 'australia': 10.0,
 'band brother': 13.0,
 'barack obama': 10.0,
 'battlestar galactica': 70.0,
 'beatles': 25.0,
 'big bang theory': 46.0,
 'blair waldorf': 11.0,
 'bleach': 11.0,
 'bone': 83.0,
 'book': 81.0,
 'booth / brennan': 17.0,
 'breaking bad': 14.0,
 'broadway': 16.0,
 'brother sister': 26.0,
 'bsg': 16.0,
 'buffy': 25.0,
 'buffy vampire slayer': 55.0,
 'burn notice': 15.0,
 'canada': 12.0,
 'castle': 42.0,
 'cat': 27.0,
 'cate blanchett': 13.0,
 'charmed': 15.0,
 'chocolate': 21.0,
 'chuck': 44.0,
 'cinema': 10.0,
 'closer': 19.0,
 'coffee': 30.0,
 'colbert report': 14.0,
 'cold case': 14.0,
 'coldplay': 16.0,
 'colin firth': 14.0,
 'community': 35.0,


Now you have two alternatives:
1. save the network to an external file and switch to interactive software **Gephi** for further network analysis. 
2. use Python's **community** module that will let you stay in the same program for the entire analysis cycle.

If you decide for 2, you can use Gephi only for visualization purposes.

In [14]:
partition = community.best_partition(tag_network)
print("Modularity: {}".format(community.modularity(partition,
                              tag_network)))

Modularity: 0.1556025462810989


Modularity of this new network is quite poor. Apparently, counting raw co-occurrences is not the best way to describe similarities — indeed, correlation-based networks are much more flexible, and you will learn about them later. 

However, even the coarse network that you have allows some meaningful interpretation.

The partition that you extracted precisely defines the term communities. (add attribute _part_ to the network nodes.) 

In [15]:
nx.set_node_attributes(tag_network, partition, "part")

The constructed network with the added attributes is your first fascinating result; 
save it without hesitation into a GraphML file in a specially created directory **results**

In [16]:
if not os.path.isdir("results"):
    os.mkdir("results")

with open("results/" + DOMAIN_NAME + ".graphml", "wb") as ofile:
    nx.write_graphml(tag_network, ofile)

Now you want a good visualization. Switch to **Gephi** to create a figure showing the whole network of tags. The node diameter must represent the number of times the corresponding tag was mentioned in the corpus, the node colors match the term communities.

## Assign names to communities

You extracted the communities, but they are nameless. You could have explored them with your bare eyes and come up with some proper names. Add other seven lines of code that locate the five most frequently used terms per cluster. Hopefully, they indeed describe the content.

Define a helper function **describe_cluster(terms_df)**. The function takes a **DataFrame** of terms in one community, extracts the namesake rows from the original **domain**, calculates their use frequency, and returns the top **HOW_MANY** performers.

In [17]:
HOW_MANY = 5
def describe_cluster(terms_df):
    # terms_df is a DataFrame; select the matching rows from "domain"
    rows = domain.join(terms_df, how="inner")
    # Calculate row sums, sort them, get the last HOW_MANY
    top_N = rows.sum(axis=1).sort_values(ascending=False)[:HOW_MANY]
    # What labels do they have?
    return top_N.index.values

Finally, convert the partition into a DataFrame, group the rows by their partition ID, and call the helper function to come up with a name for each community.

In [18]:
tag_clusters = pd.DataFrame({"part_id" : pd.Series(partition)})
results = tag_clusters.groupby("part_id").apply(describe_cluster)
for r in results:
  print("-- {}".format("; ".join(r.tolist())))

-- lost; veronica mar; battlestar galactica; glee; supernatural
-- bone; grey ' anatomy; house; gilmore girl; friend
-- good wife; harry potter; doctor; firefly; jane austen
-- music; reading; movie; writing; fanfiction
-- west wing; x - file; hugh laurie; house md; lisa edelstein


## Interpret the results

At the lower level, you can conclude that all 319 terms selected for analysis are important for **The Good Wife** viewers — otherwise, the viewers would not have selected them. Moreover, some of the terms go together more often than the others. You can make mechanistic conclusions without having the slightest idea about the cultural domain.

At the higher level, you might be an ethnographer, anthropologist, psychologist or sociologist interested in the mindscape of **The Good Wife** fans. In particular, you might want to compare their mindscape to the mindscapes of, say, **Harry Potter** or **House M.D.** fans. However, since you are a humble computer programmer or data scientist, you shall entrust the higher level interpretation to the SMEs.